# Cloud Storage Optimization — Data Analyst Project
This notebook analyzes cloud storage logs to identify **cold data**, recommend archival actions, and estimate **potential AWS S3 cost savings**.

**Deliverables**: EDA, optimization model, visuals, and exportable summary tables.


## 1. Setup & Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

pd.set_option('display.max_columns', 50)
print('Libraries loaded.')

## 2. Load Dataset

In [ ]:
DATA_PATH = 'storage_logs.csv'  # Ensure this file is in the same folder
df = pd.read_csv(DATA_PATH)
df['Last_Access_Date'] = pd.to_datetime(df['Last_Access_Date'], errors='coerce')
print(df.shape)
df.head()

## 3. Quick Data Quality Checks

In [ ]:
missing = df.isna().sum()
duplicates = df.duplicated().sum()
print('Missing values by column:\n', missing)
print('\nDuplicate rows:', duplicates)
df.describe(include='all')

## 4. Feature Engineering — Access Age & Tier Costs

In [ ]:
TODAY = pd.Timestamp.today().normalize()
df['Days_Since_Access'] = (TODAY - df['Last_Access_Date']).dt.days
tier_cost = {'S3 Standard': 0.023, 'S3 IA': 0.0125, 'Glacier': 0.004}
df['Cost_per_GB'] = df['Storage_Tier'].map(tier_cost)
df['Current_Monthly_Cost'] = (df['Size_GB'] * df['Cost_per_GB']).round(4)
df.head()

## 5. Define Cold Data & Potential Savings Model
- **Cold data**: `Days_Since_Access > 180`
- If cold and not already in Glacier, propose **migrate to Glacier**.
- Savings per file = `Current cost - Glacier cost`.

In [ ]:
glacier_cost_per_gb = 0.004
df['Is_Cold'] = df['Days_Since_Access'] > 180
df['Proposed_Tier'] = np.where(df['Is_Cold'] & (df['Storage_Tier'] != 'Glacier'), 'Glacier', df['Storage_Tier'])
df['Proposed_Cost'] = (df['Size_GB'] * np.where(df['Proposed_Tier']=='Glacier', glacier_cost_per_gb, df['Cost_per_GB'])).round(4)
df['Potential_Savings'] = (df['Current_Monthly_Cost'] - df['Proposed_Cost']).clip(lower=0).round(4)
df[['File_ID','Department','Storage_Tier','Is_Cold','Size_GB','Current_Monthly_Cost','Proposed_Tier','Proposed_Cost','Potential_Savings']].head()

## 6. Portfolio-Ready KPIs

In [ ]:
total_cost = df['Current_Monthly_Cost'].sum()
potential_savings = df['Potential_Savings'].sum()
cold_ratio = df['Is_Cold'].mean()
kpis = pd.DataFrame({
    'Metric': ['Total Current Monthly Cost (USD)', 'Total Potential Monthly Savings (USD)', 'Cold Data Ratio'],
    'Value': [round(total_cost,2), round(potential_savings,2), round(cold_ratio*100,2)]
})
kpis

## 7. Department-Level Summary

In [ ]:
dept_summary = df.groupby('Department').agg(
    Files=('File_ID','count'),
    Size_GB=('Size_GB','sum'),
    Current_Cost=('Current_Monthly_Cost','sum'),
    Potential_Savings=('Potential_Savings','sum'),
    Cold_Share=('Is_Cold','mean')
).reset_index()
dept_summary['Cold_Share'] = (dept_summary['Cold_Share']*100).round(2)
dept_summary.sort_values('Potential_Savings', ascending=False)

## 8. File-Type Summary

In [ ]:
type_summary = df.groupby('File_Type').agg(
    Files=('File_ID','count'),
    Size_GB=('Size_GB','sum'),
    Current_Cost=('Current_Monthly_Cost','sum'),
    Potential_Savings=('Potential_Savings','sum'),
    Cold_Share=('Is_Cold','mean')
).reset_index()
type_summary['Cold_Share'] = (type_summary['Cold_Share']*100).round(2)
type_summary.sort_values('Potential_Savings', ascending=False)

## 9. Visualizations

In [ ]:
# 9.1 Current cost by storage tier
tier_costs = df.groupby('Storage_Tier')['Current_Monthly_Cost'].sum().sort_values()
plt.figure()
tier_costs.plot(kind='bar')
plt.title('Current Monthly Cost by Storage Tier')
plt.ylabel('USD')
plt.xlabel('Storage Tier')
plt.tight_layout()
plt.show()

In [ ]:
# 9.2 Potential savings by department
dept_savings = df.groupby('Department')['Potential_Savings'].sum().sort_values()
plt.figure()
dept_savings.plot(kind='bar')
plt.title('Potential Monthly Savings by Department')
plt.ylabel('USD')
plt.xlabel('Department')
plt.tight_layout()
plt.show()

In [ ]:
# 9.3 Distribution of days since last access
plt.figure()
df['Days_Since_Access'].dropna().plot(kind='hist', bins=30)
plt.title('Distribution: Days Since Last Access')
plt.xlabel('Days')
plt.ylabel('Frequency')
plt.tight_layout()
plt.show()

## 10. Actionable Recommendations
- Archive cold data (>180 days) not already in Glacier.
- Review **Media** and **Backup** file types with high cold share for lifecycle policies.
- Implement S3 Lifecycle rules to auto-transition to Glacier after 180 days.
- Track KPIs monthly: total cost, cold share, realized savings.

## 11. Export Results

In [ ]:
export_dir = 'outputs'
import os
os.makedirs(export_dir, exist_ok=True)
kpis.to_csv(os.path.join(export_dir,'kpis_summary.csv'), index=False)
dept_summary.to_csv(os.path.join(export_dir,'department_summary.csv'), index=False)
type_summary.to_csv(os.path.join(export_dir,'filetype_summary.csv'), index=False)
recommendations = df.loc[(df['Is_Cold']) & (df['Storage_Tier']!='Glacier'),                          ['File_ID','Department','File_Type','Size_GB','Days_Since_Access','Storage_Tier','Proposed_Tier','Potential_Savings']]
recommendations.sort_values('Potential_Savings', ascending=False).to_csv(os.path.join(export_dir,'file_level_recommendations.csv'), index=False)
print('Exports written to', export_dir)

## 12. Next Steps (Optional Enhancements)
- Add **forecasting** for storage growth.
- Incorporate **S3 retrieval costs** and **request costs** for a fuller TCO model.
- Build a Power BI/Tableau dashboard from the exported CSV summaries.
